In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from scipy.stats import kstest, ks_2samp, t, normaltest, norm, chisquare
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from distribution import Distribution
from feature_matrix import FeatureMatrix
from data_validator import DataValidator
import metrics
import logging

logging.basicConfig(
     level=logging.INFO, 
     format= '%(asctime)s : %(levelname)s : %(message)s'
     )
logger = logging.getLogger()
 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
data = pd.read_csv("https://raw.githubusercontent.com/erdemsirel/ozu_data_science/master/M4%20-%20Applied%20Machine%20Learning%20-%20II/Call%20Details-Data.csv")
data.head(1)

,Phone Number,Account Length,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn
0,382-4657,128,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.7,1,False


In [5]:
data1, data2 = train_test_split(data, test_size=0.33, random_state=35)

In [6]:
dv = DataValidator(bins=10, only_unsuccessfull=False)
dv.fit(data1)
dv.to_pickle("test.pkl")

dv = DataValidator.read_pickle("test.pkl")

metrics_1 = dv.transform(data2)

data2_ = data2.copy()
data2_["Day Mins"] = np.random.random(size=len(data2_))
metrics_2 = dv.transform(data2_)
# dv.feature_matrix.plot_attr(attr="pdf_")

2022-05-11 00:17:58,754 : WARNING : Cannot create Distribution object for VMail Message.
2022-05-11 00:17:59,054 : WARNING : Cannot create Distribution object for Intl Calls.
2022-05-11 00:17:59,090 : WARNING : Cannot create Distribution object for CustServ Calls.
2022-05-11 00:17:59,100 : INFO : The DataValidator instance has been saved as pickle into test.pkl path.
2022-05-11 00:17:59,117 : INFO : The DataValidator instance has been loaded from test.pkl path.
/Users/dh05ju/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


(-90.791, 11129.381]       0.98778
(11129.381, 22238.462]     0.01222
(22238.462, 33347.543]     0.00000
(33347.543, 44456.624]     0.00000
(44456.624, 55565.705]     0.00000
(55565.705, 66674.786]     0.00000
(66674.786, 77783.867]     0.00000
(77783.867, 88892.948]     0.00000
(88892.948, 100002.029]    0.00000
(100002.029, 111111.11]    0.00000
Name: Night Mins, dtype: float64
(-90.791, 11129.381]       0.988375
(11129.381, 22238.462]     0.011566
(22238.462, 33347.543]     0.000000
(33347.543, 44456.624]     0.000000
(44456.624, 55565.705]     0.000000
(55565.705, 66674.786]     0.000000
(66674.786, 77783.867]     0.000000
(77783.867, 88892.948]     0.000000
(88892.948, 100002.029]    0.000000
(100002.029, 111111.11]    0.000059
Name: Night Mins, dtype: float64


2022-05-11 00:18:46,777 : INFO : Feature Matrix metrics
|                | chisquare_pvalue   | kstest_pvalue   | missing_val_rat_rel_diff   | psi     | rel_abs_diff_of_means   | z_score_abs   |   failed_test_count | success   |
|:---------------|:-------------------|:----------------|:---------------------------|:--------|:------------------------|:--------------|--------------------:|:----------|
| Night Mins     | ✓ 1.0              | ✓ 0.0           | ✓ 0.0                      | X inf   | ✓ 0.012                 | ✓ 0.003       |                   1 | X         |
| Account Length | ✓ 1.0              | ✓ 0.0           | ✓ 0.0                      | ✓ 0.0   | ✓ 0.005                 | ✓ 0.001       |                   0 | ✓         |
| Day Mins       | ✓ 1.0              | ✓ 0.0           | ✓ 0.0                      | ✓ 0.0   | ✓ 0.058                 | ✓ 0.016       |                   0 | ✓         |
| Day Calls      | ✓ 1.0              | ✓ 0.0           | ✓ 0.0                

(-111.111, 11111.111]     1.0
(11111.111, 22222.222]    0.0
(22222.222, 33333.333]    0.0
(33333.333, 44444.444]    0.0
(44444.444, 55555.555]    0.0
(55555.555, 66666.666]    0.0
(66666.666, 77777.777]    0.0
(77777.777, 88888.888]    0.0
(88888.888, 99999.999]    0.0
(99999.999, 111111.11]    0.0
Name: Day Mins, dtype: float64
(-111.111, 11111.111]     0.990662
(11111.111, 22222.222]    0.009323
(22222.222, 33333.333]    0.000000
(33333.333, 44444.444]    0.000000
(44444.444, 55555.555]    0.000000
(55555.555, 66666.666]    0.000000
(66666.666, 77777.777]    0.000000
(77777.777, 88888.888]    0.000000
(88888.888, 99999.999]    0.000000
(99999.999, 111111.11]    0.000015
Name: Day Mins, dtype: float64


/Users/dh05ju/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


(-90.791, 11129.381]       0.98778
(11129.381, 22238.462]     0.01222
(22238.462, 33347.543]     0.00000
(33347.543, 44456.624]     0.00000
(44456.624, 55565.705]     0.00000
(55565.705, 66674.786]     0.00000
(66674.786, 77783.867]     0.00000
(77783.867, 88892.948]     0.00000
(88892.948, 100002.029]    0.00000
(100002.029, 111111.11]    0.00000
Name: Night Mins, dtype: float64
(-90.791, 11129.381]       0.988375
(11129.381, 22238.462]     0.011566
(22238.462, 33347.543]     0.000000
(33347.543, 44456.624]     0.000000
(44456.624, 55565.705]     0.000000
(55565.705, 66674.786]     0.000000
(66674.786, 77783.867]     0.000000
(77783.867, 88892.948]     0.000000
(88892.948, 100002.029]    0.000000
(100002.029, 111111.11]    0.000059
Name: Night Mins, dtype: float64


2022-05-11 00:19:35,542 : INFO : Feature Matrix metrics
|                | chisquare_pvalue   | kstest_pvalue   | missing_val_rat_rel_diff   | psi     | rel_abs_diff_of_means   | z_score_abs   |   failed_test_count | success   |
|:---------------|:-------------------|:----------------|:---------------------------|:--------|:------------------------|:--------------|--------------------:|:----------|
| Day Mins       | ✓ 0.508            | ✓ 0.0           | ✓ 0.0                      | X inf   | ✓ 0.999                 | ✓ 0.278       |                   1 | X         |
| Night Mins     | ✓ 1.0              | ✓ 0.0           | ✓ 0.0                      | X inf   | ✓ 0.012                 | ✓ 0.003       |                   1 | X         |
| Account Length | ✓ 1.0              | ✓ 0.0           | ✓ 0.0                      | ✓ 0.0   | ✓ 0.005                 | ✓ 0.001       |                   0 | ✓         |
| Day Calls      | ✓ 1.0              | ✓ 0.0           | ✓ 0.0                

In [7]:
metrics_2

,chisquare_pvalue,chisquare_pvalue_result,kstest_pvalue,kstest_pvalue_result,missing_val_rat_rel_diff,missing_val_rat_rel_diff_result,psi,psi_result,rel_abs_diff_of_means,rel_abs_diff_of_means_result,z_score_abs,z_score_abs_result,failed_test_count,success
Day Mins,0.507758,True,0.0,True,0.0,True,inf,False,0.999134,True,0.277901,True,1,False
Night Mins,1.000000,True,0.0,True,0.0,True,inf,False,0.011569,True,0.003109,True,1,False
Account Length,1.000000,True,0.0,True,0.0,True,4.199568e-07,True,0.004920,True,0.001122,True,0,True
Day Calls,1.000000,True,0.0,True,0.0,True,5.682821e-05,True,0.015469,True,0.004454,True,0,True
Day Charge,1.000000,True,0.0,True,0.0,True,5.842725e-05,True,0.002328,True,0.001131,True,0,True
Eve Mins,1.000000,True,0.0,True,0.0,True,1.212549e-05,True,0.002364,True,0.000621,True,0,True
Eve Calls,0.999999,True,0.0,True,0.0,True,5.538117e-05,True,0.014445,True,0.004082,True,0,True
Eve Charge,1.000000,True,0.0,True,0.0,True,2.709080e-05,True,0.000479,True,0.000454,True,0,True
Night Calls,1.000000,True,0.0,True,0.0,True,2.765973e-05,True,0.015959,True,0.004783,True,0,True
Night Charge,1.000000,True,0.0,True,0.0,True,1.113510e-04,True,0.000405,True,0.000346,True,0,True


In [ ]:
np.round((0.01 / (0.01 +0.00001)), 3)

In [ ]:
a = metrics_2[[col for col in metrics_2.columns if not col.endswith("_result")]].round(3)
logger.info("\n"+a.to_markdown(), extra=a.to_dict())

In [ ]:
c = [col for col in metrics_2.columns if not col.endswith("_result")]
for col in c:
    if col+"_result" not in metrics_2.columns: continue
    metrics_2[col] = metrics_2[col+"_result"].replace({True: "✓", False:"X"}) + " " +  metrics_2[col].round(3).astype(str)

In [ ]:
metrics_2

In [ ]:
(~metrics_2[[col for col in metrics_2.columns if col.endswith("_result")]]).sum(axis=1)

In [ ]:
metrics_.join(a, rsuffix='_result')

In [ ]:
kstest(X_test.mileage, X_train.mileage)

In [ ]:
kstest(X_test.mileage, lambda x: metrics.cdf(x, X_train.mileage, bins=100))


In [ ]:
a = np.random.normal(X_train.mileage.mean(), X_train.mileage.std(), size=len(X_train.mileage))
kstest(a, lambda x: metrics.cdf(x, X_train.mileage, bins=100))

In [ ]:
kstest(a, X_train.mileage)

In [ ]:
X_train.mileage.value_counts(bins=5, normalize=True).sort_index().cumsum()

In [ ]:
%load_ext autoreload
%autoreload 2

import metrics

In [ ]:
ref_var = X_train.engineSize
var = X_test.engineSize
print(metrics.abs_difference_of_means(ref_var, var))
print(metrics.relative_abs_difference_of_means(ref_var, var))
print(metrics.relative_wasserstein_varance(ref_var, var))
print(metrics.is_var_within_z_threshold(ref_var, var, z_threshold=1))
print(metrics.is_ks_successfull(ref_var, var))
print(metrics.is_vars_normal(ref_var, var))
print(metrics.psi(ref_var, var))


In [ ]:
X_train.describe()

In [ ]:
X_test.describe()

In [ ]:
ref_var.hist()

In [ ]:
ref_dist, ref_dist_bins = pd.cut(ref_var, 10, retbins=True)
ref_dist = ref_dist.value_counts(normalize=True)

In [ ]:
var_dist = pd.cut(var, bins=ref_dist_bins).value_counts(normalize=True)

In [ ]:
PSI=((var_dist - ref_dist) * np.log(var_dist / ref_dist)).sum()
PSI

In [ ]:
np.log(var_dist / ref_dist)

In [ ]:
ref_dist, ref_dist_bins = pd.qcut(ref_var, 4, retbins=True)
ref_dist = ref_dist.value_counts(normalize=True)

var_dist = pd.cut(var, bins=ref_dist_bins).value_counts(normalize=True)
var_dist

In [ ]:
def cdf(x, X):
    pd.qcut(ref_var, 4, retbins=True)

In [ ]:
ref_var.value_counts(bins=10, normalize=True).sort_index().cumsum().loc[2]

In [ ]:
pdf_ = ref_var.value_counts(bins=10, normalize=True).sort_index().cumsum()
a.iloc[a.index.get_loc(5)]